In [1]:
import pandas as pd
from nltk.corpus import stopwords
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessingStopwords
from contextualized_topic_models.models.ctm import ZeroShotTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation

In [7]:
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", category = DeprecationWarning)
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
stop_words = stopwords.words('english')
stop_words.append("said")
bbc_df = pd.read_csv("../data/bbc-text.csv")

           category                                               text
0              tech  tv future in the hands of viewers with home th...
1          business  worldcom boss  left books alone  former worldc...
2             sport  tigers wary of farrell  gamble  leicester say ...
3             sport  yeading face newcastle in fa cup premiership s...
4     entertainment  ocean s twelve raids box office ocean s twelve...
...             ...                                                ...
2220       business  cars pull down us retail figures us retail sal...
2221       politics  kilroy unveils immigration policy ex-chatshow ...
2222  entertainment  rem announce new glasgow concert us band rem h...
2223       politics  how political squabbles snowball it s become c...
2224          sport  souness delight at euro progress boss graeme s...

[2225 rows x 2 columns]


In [4]:
documents = bbc_df["text"]
preprocessor = WhiteSpacePreprocessingStopwords(documents, stopwords_list=stop_words) 
preprocessed_documents, unpreprocessed_documents, vocab, indices = preprocessor.preprocess() 

In [8]:
tp = TopicModelDataPreparation("distiluse-base-multilingual-cased")
training_dataset = tp.fit(text_for_contextual=unpreprocessed_documents, text_for_bow=preprocessed_documents)

/home/zhenya/.cache/pypoetry/virtualenvs/book-2nd-ed-tI3JII_H-py3.10/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/zhenya/.cache/pypoetry/virtualenvs/book-2nd-ed-tI3JII_H-py3.10/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/zhenya/.cache/pypoetry/virtualenvs/book-2nd-ed-tI3JII_H-py3.10/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm =

Batches:   0%|          | 0/12 [00:00<?, ?it/s]

In [6]:
ctm = ZeroShotTM(bow_size=len(tp.vocab), contextual_size=512, n_components=5, num_epochs=100)
ctm.fit(training_dataset)

Epoch: [100/100]	 Seen Samples: [217600/222500]	Train Loss: 1070.975986256319	Time: 0:00:00.437354: : 100it [00:45,  2.19it/s]
100%|███████████████████████████████████████████████████████████████████████████████████| 35/35 [00:00<00:00, 87.82it/s]


In [9]:
ctm.get_topics()

defaultdict(list,
            {0: ['people',
              'music',
              'technology',
              'users',
              'mobile',
              'digital',
              'tv',
              'games',
              'net',
              'broadband'],
             1: ['match',
              'injury',
              'side',
              'back',
              'england',
              'win',
              'victory',
              'goal',
              'club',
              'great'],
             2: ['analysts',
              'bought',
              'oil',
              'figures',
              'warned',
              'value',
              'securities',
              'october',
              'payments',
              'analyst'],
             3: ['film',
              'best',
              'actress',
              'award',
              'director',
              'awards',
              'stars',
              'band',
              'oscars',
              'album'],
             4: ['

In [10]:
spanish_news_piece = """IBM anuncia el comienzo de la “era de la utilidad cuántica” y anticipa un superordenador en 2033. 
La compañía asegura haber alcanzado un sistema de computación que no se puede simular con procedimientos clásicos."""
testing_dataset = tp.transform([spanish_news_piece])

/home/zhenya/.cache/pypoetry/virtualenvs/book-2nd-ed-tI3JII_H-py3.10/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/zhenya/.cache/pypoetry/virtualenvs/book-2nd-ed-tI3JII_H-py3.10/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/zhenya/.cache/pypoetry/virtualenvs/book-2nd-ed-tI3JII_H-py3.10/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm =

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
ctm.get_doc_topic_distribution(testing_dataset)

100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.97it/s]


array([[0.5902461 , 0.09361929, 0.14041995, 0.07586181, 0.0998529 ]],
      dtype=float32)